<a href="https://colab.research.google.com/github/djliden/numerai_starter_kit/blob/main/Numerai_Starter_Kit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
This notebook will walk you through the entire process of making a [numerai](numer.ai) submission, from downloading the data to submitting final predictions, all in a google colab notebook. In particular, it will address two challenges:
- handling API keys in a remote environment (colab)
- parsing the large CSV files which, if read all at once, will exceed colab's memory and cause the notebook to crash.

This notebook will implement two models: a basic tabular neural network using `fastai` and a linear regression model using `scikit-learn`.

# Initial Setup
First, we install and import the necessary packages. If you don't care about all of the resulting messages, uncomment # %%capture at the top of the cell; this will hide all of the output. 

In [9]:
# %%capture
# install
!pip install --upgrade python-dotenv fastai numerapi

# import dependencies
import os
from dotenv import load_dotenv, find_dotenv
from getpass import getpass
import pandas as pd
import numpy as np
import numerapi
from fastai.tabular.all import *

Requirement already up-to-date: python-dotenv in /usr/local/lib/python3.7/dist-packages (0.15.0)
Requirement already up-to-date: fastai in /usr/local/lib/python3.7/dist-packages (2.2.7)
Requirement already up-to-date: numerapi in /usr/local/lib/python3.7/dist-packages (2.4.3)


# Setting Up numerapi
We will use the [numerapi](https://github.com/uuazed/numerapi) package to access the data and make submissions. For this to work, numerapi needs to use your API keys (which can be obtained [here](https://numer.ai/submit)). We will set up two main ways of passing these API keys to a numerapi instance:
1. Read a `.env` file using the `python-dotenv` package. This will require you to upload a `.env` file (which contains your secret key and should *not* be kept under version control). Using this method means you will not have to directly enter your keys each time you use this notebook, though you will need to re-upload the `.env` file.
2. Manually entering the API keys -- if you don't have access to, or don't want to mess with, your `.env` file.

If you have a `.env` file, upload it (to the default working directory, `content`, now. In either case, run the cell below to set up the numerapi instance. See [Appendix A](#app_a) for instructions on generating and downloading a .env file.

In [ ]:
# Load the numerapi credentials from .env or prompt for them if not available
def credential():
    dotenv_path = find_dotenv()
    load_dotenv(dotenv_path)

    if os.getenv("NUMERAI_PUBLIC_KEY"):
        print("Loaded Numerai Public Key into Global Environment!")
    else:
        os.environ["NUMERAI_PUBLIC_KEY"] = getpass("Please enter your Numerai Public Key. You can find your key here: https://numer.ai/submit -> ")
    
    if os.getenv("NUMERAI_SECRET_KEY"):
        print("Loaded Numerai Secret Key into Global Environment!")
    else:
        os.environ["NUMERAI_SECRET_KEY"] = getpass("Please enter your Numerai Secret Key. You can find your key here: https://numer.ai/submit -> ")

public_key = os.environ.get("NUMERAI_PUBLIC_KEY")
secret_key = os.environ.get("NUMERAI_SECRET_KEY")
napi = numerapi.NumerAPI(verbosity="info", public_id=public_key, secret_key=secret_key)

<a name="app_a"></a>
# Appendix A
## Generating and Saving a `.env` file
Note: if you run the below section, you should *not* save an edited version of this notebook where it can be accessed by others. You do not want to share your API keys with others. I recommend filling out this section to generate a `.env` file and then downloading that file for future use.

In [10]:
# Write lines to file

with open('./.env', 'w') as dotenv:
    dotenv.write(f'NUMERAI_PUBLIC_KEY = {getpass("Please enter your Numerai Public Key. You can find your key here: https://numer.ai/submit -> ")}\n')
    dotenv.write("NUMERAI_SECRET_KEY = YOUR_SECRET_KEY_HERE\n")
    dotenv.write("NUMERAI_MODEL_ID = YOUR_MODEL_ID\n")

Please enter your Numerai Public Key. You can find your key here: https://numer.ai/submit -> ··········


In [11]:
!cat .env

NUMERAI_PUBLIC_KEY = X5ACOQPPFFV7SK2H2CBVX53GHWLYPDCB
NUMERAI_SECRET_KEY = YOUR_SECRET_KEY_HERE
NUMERAI_MODEL_ID = YOUR_MODEL_ID
